# API CONNECTIONS

# 1 Post Images

## 1.1 Import libraries & credentials

In [1]:
from imagekitio import ImageKit
import json
import base64
import requests
import datetime

from sqlalchemy import create_engine, text

In [60]:
# Specify the path to your credentials file
credentials_file_path = "/Users/miguelgranica/Documents/MBIT - DE/Modulo 02 - Arquitecturas transaccionales/M02 S05 Proyecto de Consolidacion/app/credentials.json"

# Load credentials from the JSON file
with open(credentials_file_path, 'r') as file:
    credentials = json.load(file)

## 1.2 Generate public URL

In [28]:
imagekit = ImageKit(
    public_key=credentials["imagekit"]["public_key"],
    private_key=credentials["imagekit"]["private_key"],
    url_endpoint =credentials["imagekit"]["url_endpoint"]
)

In [29]:
image_path = "../static/images/3005501.jpg"
with open(image_path, mode="rb") as img:
    imgstr = base64.b64encode(img.read())

In [30]:
# upload an image
upload_info = imagekit.upload(file=imgstr, file_name="my_file_name.jpg")

In [31]:
type(upload_info)

imagekitio.models.results.UploadFileResult.UploadFileResult

In [32]:
image_url = upload_info.url
image_url

'https://ik.imagekit.io/83tu2lzrih/my_file_name_qTAjoBH_y.jpg'

In [33]:
type(upload_info.file_id)


str

## 1.3 Tag Extraction

In [34]:
api_key = credentials["imagga"]["api_key"]
api_secret = credentials["imagga"]["api_secret"]
image_url = image_url
# 
min_confidence = 80

In [35]:
response = requests.get(f"https://api.imagga.com/v2/tags?image_url={image_url}", auth=(api_key, api_secret))

In [37]:
tags = [
    {
        "tag": t["tag"]["en"],
        "confidence": t["confidence"]
    }
    for t in response.json()["result"]["tags"]
    if t["confidence"] > min_confidence
]

In [14]:
tags

[{'tag': 'jersey', 'confidence': 93.545166015625},
 {'tag': 'shirt', 'confidence': 92.3850555419922}]

## 1.4 Delete public URL

In [38]:
# delete an image
delete_image = imagekit.delete_file(file_id=upload_info.file_id)
delete_image

In [40]:
# Final Result
print()

# Raw Response
print(delete_image.response_metadata.raw)


None


## 1.5 File Storage

## 1.6 Database Storage

- En la tabla `pictures` almacenaremos el path de la imagen que tendrá un id asociado. También almacenaremos la fecha en la que se ha introducido como un string en formato `YYYY-MM-DD HH:MM:SS`.
- En la tabla `tags` almacenaremos una fila por cada tag asociado a la imagen, donde tendremos los campos `tag`, `picture_id` (foreign key) y la fecha en la que se ha introducido la imagen como un string en formato `YYYY-MM-DD HH:MM:SS`. También se almacenará qué `confidence` tiene esta tag para la imagen.

In [2]:
# Pictures database
host = 'localhost'
database = 'Pictures'
user = 'mbit'
password = 'mbit'

In [3]:
# create sqlalchemy engine
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/{database}")

In [20]:
str(datetime.datetime.now())

'2023-12-27 18:40:13.469287'

In [35]:
pictures_table = "pictures"

In [22]:
insert_stmt = f"INSERT INTO {pictures_table} (id, path, size, date) VALUES ('{upload_info.file_id}', '{upload_info.file_path}', '{upload_info.size}', '{str(datetime.datetime.now())}');"
insert_stmt

"INSERT INTO pictures (id, path, size, date) VALUES ('658c615a88c257da333630c3', '/my_file_name_NLDiFA117.jpg', '59898', '2023-12-27 18:40:59.190548');"

In [23]:
with engine.connect() as conn:
    conn.execute(text(insert_stmt))
    conn.commit()

In [36]:
with engine.connect() as conn:
    result=conn.execute(text("SELECT * FROM pictures"))
    columns = result.keys()
    data = [
        dict(zip(columns, row))
        for row in result
    ]

In [37]:
data

[{'id': '658c615a88c257da333630c3',
  'path': '/my_file_name_NLDiFA117.jpg',
  'size': 59898,
  'date': '2023-12-27 18:40:59.190548'},
 {'id': '658c768088c257da3396084a',
  'path': '/my_file_name_4Ti0Hwbt0.jpg',
  'size': 260284,
  'date': '2023-12-27 20:09:51.595350'}]

In [38]:
tags_table = "tags"

In [29]:
for tag in tags:
    tag, confidence = tag["tag"], tag["confidence"]
    insert_stmt = f"INSERT INTO {tags_table} (tag, picture_id, confidence, date) VALUES ('{tag}', '{upload_info.file_id}', '{confidence}', '{datetime.datetime.now()}');"
    with engine.connect() as conn:
        conn.execute(text(insert_stmt))
        conn.commit()

In [40]:
with engine.connect() as conn:
    result=conn.execute(text("SELECT * FROM tags"))
    columns = result.keys()
    data = [
        dict(zip(columns, row))
        for row in result
    ]

In [41]:
data

[{'tag': 'architecture',
  'picture_id': '658c768088c257da3396084a',
  'confidence': 98,
  'date': '2023-12-27 20:09:51.595350'},
 {'tag': 'jersey',
  'picture_id': '658c615a88c257da333630c3',
  'confidence': 94,
  'date': '2023-12-27 18:41:29.786316'},
 {'tag': 'palace',
  'picture_id': '658c768088c257da3396084a',
  'confidence': 77,
  'date': '2023-12-27 20:09:51.595350'},
 {'tag': 'shirt',
  'picture_id': '658c615a88c257da333630c3',
  'confidence': 92,
  'date': '2023-12-27 18:41:29.800547'}]

##

# 2. GET Images

#### GET images

Este endpoint sirve para obtener una lista de imágenes que cumplan un filtro. Se proporcionará mediante _query parameters_ la siguiente información:

- `min_date`/`max_date`: opcionalmente se puede indicar una fecha mínima y máxima, en formato `YYYY-MM-DD HH:MM:SS`, para obtener imágenes cuya fecha de registro esté entre ambos valores. Si no se proporciona `min_date` no se filtrará ninguna fecha inferiormente. Si no se proporciona `max_date` no se filtrará ninguna fecha superiormente.

- `tags`: optionalmente se puede indicar una lista de tags. Las imágenes devueltas serán aquellas que incluyan **todas** las tags indicadas. El formato de este campo será un string donde las tags estarán separadas por comas, por ejemplo `"tag1,tag2,tag3"`. Si no se proporciona ninguna tag, no se devolverá ninguna imagen.

La **respuesta** del endpoint será una lista de objetos imagen con los siguientes campos:

- `id`: identificador de la imagen
- `size`: tamaño de la imagen en KB
- `date`: fecha en la que se registró la imagen, en formato `YYYY-MM-DD HH:MM:SS`
- `tags`: lista de objetos identificando las tags asociadas a la imágen. Cada objeto tendrá el siguiente formato:
    - `tag`: nombre de la tag
    - `confidence`: confianza con la que la etiqueta está asociada a la imagen

## 2.1 Parameters

In [19]:
min_date = '2023-12-27 18:45:59'
max_date = None #datetime.date(2023, 12, 18)
tags = ['architecture', 'palace']

In [8]:
all([min_date, max_date])

False

## 2.2 Get Images

In [15]:
def get_images(min_date=None, max_date=None , tags=None):
    sql_query = """
        SELECT p.id, p.size, p.date,
            GROUP_CONCAT(t.tag) AS tags,
            GROUP_CONCAT(t.confidence) AS confidences
        FROM pictures p
        JOIN tags t ON p.id = t.picture_id
        WHERE 1=1
    """

    # Conditionally filters based on parameters
    params = {}
    if min_date is not None:
        sql_query += " AND p.date > :min_date"
        params["min_date"] = min_date
    if max_date is not None:
        sql_query += " AND p.date < :max_date"
        params["max_date"] = max_date
    if tags is not None:
        sql_query += " AND t.tags IN :tags"
        params["tags"]: ", ".join(tags)

    # Group by id, date, and size
    sql_query += " GROUP BY p.id, p.date, p.size"

    # Execute the query and return the results
    with engine.connect() as conn:
        result = conn.execute(text(sql_query), params)
    columns = result.keys()
    data = [
        {**{key: value for key, value in i.items() if key not in ["tags", "confidences"]},
         "tags": {key: value for key, value in i.items() if key in ["tags", "confidences"]}}
        for i in [ 
            dict(zip(columns, row)) for row in result
        ]
    ]
    return data

In [53]:
#Params definitions
min_date = '2023-12-27 18:45:59'
max_date = '2024-01-05 18:45:59'
tags = ["python"]

sql_query = """
        SELECT p.id, p.size, p.date,
            GROUP_CONCAT(t.tag) AS tags,
            GROUP_CONCAT(t.confidence) AS confidences
        FROM pictures p
        JOIN tags t ON p.id = t.picture_id
        WHERE 1=1
    """

    # Conditionally filters based on parameters
params = {}
if min_date is not None:
    sql_query += " AND p.date > :min_date"
    params["min_date"] = min_date
if max_date is not None:
    sql_query += " AND p.date < :max_date"
    params["max_date"] = max_date
if tags is not None:
    sql_query += " AND t.tag IN :tags"
    params["tags"] = tags

# Group by id, date, and size
sql_query += " GROUP BY p.id, p.date, p.size"

# Context manager to execute the query and return the results
with engine.connect() as conn:
    result = conn.execute(text(sql_query), params)


In [56]:
result.rowcount == 0

True

In [17]:
images = get_images()
images

[{'id': '658c615a88c257da333630c3',
  'size': 59898,
  'date': '2023-12-27 18:40:59.190548',
  'tags': {'tags': 'jersey,shirt', 'confidences': '94,92'}},
 {'id': '658c768088c257da3396084a',
  'size': 260284,
  'date': '2023-12-27 20:09:51.595350',
  'tags': {'tags': 'architecture,palace', 'confidences': '98,77'}},
 {'id': '658f24b888c257da3335e5b7',
  'size': 260284,
  'date': '2023-12-29 20:57:43.592105',
  'tags': {'tags': 'architecture,palace', 'confidences': '98,77'}},
 {'id': '659050c088c257da3341140e',
  'size': 90037,
  'date': '2023-12-30 18:17:51.852108',
  'tags': {'tags': 'microwave,stove', 'confidences': '82,100'}},
 {'id': '659051e988c257da334454ea',
  'size': 5293488,
  'date': '2023-12-30 18:22:46.410381',
  'tags': {'tags': 'beer glass,container,glass',
   'confidences': '100,100,100'}},
 {'id': '659053f288c257da334d0b93',
  'size': 5293488,
  'date': '2023-12-30 18:31:26.547299',
  'tags': {'tags': 'beer glass,container,glass',
   'confidences': '100,100,100'}},
 {'id'

# 3. Get Image

Este endpoint sirve para descargarse una imágen y sus propiedades. Se proporcionará mediante _path parameter_ el id de la imagen y se la **respuesta** será un json con los siguientes campos:

- `id`: identificador de la imagen
- `size`: tamaño de la imagen en KB
- `date`: fecha en la que se registró la imagen, en formato `YYYY-MM-DD HH:MM:SS`
- `tags`: lista de objetos identificando las tags asociadas a la imágen. Cada objeto tendrá el siguiente formato:
    - `tag`: nombre de la tag
    - `confidence`: confianza con la que la etiqueta está asociada a la imagen
- `data`: imagen como string codificado en base64

In [207]:
id = '658c615a88c257da333630c3'

In [215]:
def get_image(engine, picture_id):
    sql_query = """
        SELECT p.id, p.size, p.date,
            GROUP_CONCAT(t.tag) AS tags,
            GROUP_CONCAT(t.confidence) AS confidences
        FROM pictures p
        JOIN tags t ON p.id = t.picture_id
        WHERE p.id = :picture_id
    """
    # set params
    params = dict(picture_id=picture_id)
    # Execute the query and return the results
    with engine.connect() as conn:
        result = conn.execute(text(sql_query), params)
    # Format response
    columns = result.keys()
    response = [
        {**{key: value for key, value in i.items() if key not in ["tags", "confidences"]},
         "tags": {key: value for key, value in i.items() if key in ["tags", "confidences"]}}
        for i in [ 
            dict(zip(columns, row)) for row in result
        ]
    ]
    return response[0]
    
    

In [216]:
res = get_image(engine, id)

In [217]:
res

{'id': '658c615a88c257da333630c3',
 'size': 59898,
 'date': '2023-12-27 18:40:59.190548',
 'tags': {'tags': 'jersey,shirt', 'confidences': '94,92'}}

# 4. Get Tags

Este endpoint sirve para obtener una lista de tags (teniendo en cuenta cualquier tag asignada a una imagen registrada) que cumplan un filtro. Se proporcionará mediante _query parameters_ la siguiente información:

- `min_date`/`max_date`: opcionalmente se puede indicar una fecha mínima y máxima, en formato `YYYY-MM-DD HH:MM:SS`, para obtener imágenes cuya fecha de registro esté entre ambos valores. Si no se proporciona `min_date` no se filtrará ningúna fecha inferiormente. Si no se proporciona `max_date` no se filtrará ningúna fecha superiormente.

El endpoint devolvera una respuesta que será una lista de objetos imagen con los siguientes campos:

- `tag`: nombre de la etiqueta
- `n_images`: número de imágenes que tienen asociada esta tag
- `min_confidence`, `max_confidence`, `mean_confidence`: confianza mínima, máxima y media de esta tag para todas las imágenes con las que está asignada
las dos APIs utilizadas.

## 4.1 

In [4]:
def get_tags(min_date=None, max_date=None):
    sql_query = """
        SELECT t.tag tag,
            COUNT(DISTINCT(t.picture_id)) n_images,
            MIN(t.confidence) min_confidence,
            MAX(t.confidence) max_confidence,
            AVG(t.confidence) mean_confidence
        FROM tags t
        JOIN pictures p ON t.picture_id = p.id
        WHERE 1=1
    """

    # Conditionally filters based on parameters
    params = {}
    if min_date is not None:
        sql_query += " AND p.date > :min_date"
        params["min_date"] = min_date
    if max_date is not None:
        sql_query += " AND p.date < :max_date"
        params["max_date"] = max_date

    # Group by id, date, and size
    sql_query += " GROUP BY t.tag"

    # Execute the query and return the results
    with engine.connect() as conn:
        result = conn.execute(text(sql_query), params)
    columns = result.keys()
    data = [
        dict(zip(columns, row))
        for row in result
    ]
    return data

In [9]:
min_date = '2023-12-27 18:45:59'
max_date = '2023-12-28 18:45:59'

In [10]:
get_tags(min_date, max_date)

[{'tag': 'architecture',
  'n_images': 1,
  'min_confidence': 98,
  'max_confidence': 98,
  'mean_confidence': Decimal('98.0000')},
 {'tag': 'palace',
  'n_images': 1,
  'min_confidence': 77,
  'max_confidence': 77,
  'mean_confidence': Decimal('77.0000')}]